In [1]:
from dotenv import load_dotenv
import os
import requests


# Tải các biến môi trường từ tệp .env
load_dotenv()

# Lấy giá trị từ biến môi trường
api_key = os.getenv('X_RAPIDAPI_KEY')
api_host = os.getenv('X_RAPIDAPI_HOST')

headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": api_host
    }

In [2]:
def get_id_film():
    movies_id = set()
    url = "https://movies-api14.p.rapidapi.com/movies"
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        try:
            movies_data = response.json()
            
            if 'movies' in movies_data:
                for movie in movies_data['movies']:
                    if movie.get('_id'):
                        movies_id.add(movie['_id'])
            else:
                print("Lỗi: Dữ liệu không có khóa 'movies'.")
        except Exception as e:
            print(f"Lỗi khi xử lý dữ liệu JSON: {e}")
    else:
        print(f"Lỗi khi gọi API: {response.status_code}")
    
    return movies_id if movies_id else None

def get_info_movie(movie_id):
    movie_data = {}
    url = f"https://movies-api14.p.rapidapi.com/movie/{movie_id}"
    
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            try:
                data = response.json().get('movie')
                if data:
                    movie_data = {
                        'backdrop_path': data.get('backdrop_path'),
                        'genres': data.get('genres', []),  
                        'original_title': data.get('original_title'),
                        'overview': data.get('overview'),
                        'poster_path': data.get('poster_path'),
                        'release_date': data.get('release_date'),
                        'title': data.get('title'),
                        'vote_average': data.get('vote_average'),
                        'vote_count': data.get('vote_count'),
                        'youtube_trailer': data.get('youtube_trailer', None)  
                    }
            except Exception as e:
                print(f"Lỗi khi xử lý dữ liệu JSON: {e}")
        else:
            print(f"Lỗi khi gọi API: {response.status_code}")
    except Exception as e:
        print(f"Lỗi khi gọi API: {e}")
    
    return movie_data

In [4]:
# Đảm bảo rằng bạn đã thiết lập biến môi trường đúng
movies_data={}
if not api_key or not api_host:
    print("Lỗi: Bạn cần thiết lập X_RAPIDAPI_KEY và X_RAPIDAPI_HOST trong môi trường.")
else:
    movies_id = get_id_film()
    if movies_id:
        for id in movies_id:
            print(f'crawl data from {id}')
            data = get_info_movie(id)
            if data:
                movies_data[id] = data

    # In movies_data để kiểm tra kết quả
    print(movies_data)


crawl data from 928769
crawl data from 860425
crawl data from 736526
crawl data from 361743
crawl data from 299536
crawl data from 843794
crawl data from 453395
crawl data from 736790
crawl data from 868759
crawl data from 1114905
crawl data from 315162
crawl data from 634649
crawl data from 675353
crawl data from 619930
crawl data from 934433
crawl data from 631842
crawl data from 635302
crawl data from 873256
crawl data from 505642
crawl data from 436270
crawl data from 804150
crawl data from 1102776
crawl data from 1023803
crawl data from 118340
crawl data from 420808
crawl data from 849869
crawl data from 920143
crawl data from 985939
crawl data from 493529
crawl data from 283995
crawl data from 899294
crawl data from 774752
crawl data from 1013860
crawl data from 616037
crawl data from 768744
crawl data from 948713
crawl data from 265712
crawl data from 998771
crawl data from 829560
crawl data from 868985
crawl data from 869626
crawl data from 638974
{928769: {'backdrop_path': 'ht

In [39]:
# !pip install firebase-admin

In [3]:
import os
import firebase_admin
from firebase_admin import credentials, firestore, exceptions
from dotenv import load_dotenv

# Nạp các biến môi trường từ file .env
load_dotenv()

# Lấy đường dẫn từ biến môi trường
service_account_key = os.getenv('SERVICE_ACCOUNT_KEY')

if not service_account_key:
    raise ValueError("Biến môi trường 'SERVICE_ACCOUNT_KEY' chưa được thiết lập!")

# Kiểm tra file tồn tại
if not os.path.exists(service_account_key):
    raise FileNotFoundError(f"File không tồn tại tại: {service_account_key}")

try:
    # Kiểm tra nếu Firebase app chưa được khởi tạo
    firebase_admin.get_app()
except ValueError:  # Nếu chưa khởi tạo, tiến hành khởi tạo
    cred = credentials.Certificate(service_account_key)
    firebase_admin.initialize_app(cred)

# Khởi tạo Firestore client
db = firestore.client()

# Kiểm tra kết nối Firestore
print("Firestore đã được kết nối thành công!")


Firestore đã được kết nối thành công!


In [44]:
# Lưu dữ liệu vào Firestore
def save_movies_to_firestore(movies_data):
    for movie_id, movie in movies_data.items():  # Using movie_id as key and movie data as value
        # Dùng movie_id làm ID document trong Firestore
        movie_ref = db.collection('movies_update').document(str(movie_id))  # Use movie_id as Firestore document ID
        
        # Lưu dữ liệu của bộ phim
        movie_ref.set({
            'backdrop_path': movie.get('backdrop_path'),
            'genres': movie.get('genres', []),  
            'original_title': movie.get('original_title'),
            'overview': movie.get('overview'),
            'poster_path': movie.get('poster_path'),
            'release_date': movie.get('release_date'),
            'title': movie.get('title'),
            'vote_average': movie.get('vote_average'),
            'vote_count': movie.get('vote_count'),
            'youtube_trailer': movie.get('youtube_trailer', None)  
        })
        print(f"Movie '{movie.get('title')}' saved to Firestore.")

save_movies_to_firestore(movies_data)

Movie 'AI Love You' saved to Firestore.
Movie 'Breathless' saved to Firestore.
Movie 'Troll' saved to Firestore.
Movie 'Top Gun: Maverick' saved to Firestore.
Movie 'Avengers: Infinity War' saved to Firestore.
Movie 'JUNG_E' saved to Firestore.
Movie 'Doctor Strange in the Multiverse of Madness' saved to Firestore.
Movie 'Chupa' saved to Firestore.
Movie 'Ghosted' saved to Firestore.
Movie 'Death's Roulette' saved to Firestore.
Movie 'Puss in Boots: The Last Wish' saved to Firestore.
Movie 'Spider-Man: No Way Home' saved to Firestore.
Movie 'Sonic the Hedgehog 2' saved to Firestore.
Movie 'Narvik' saved to Firestore.
Movie 'Scream VI' saved to Firestore.
Movie 'Knock at the Cabin' saved to Firestore.
Movie 'Demon Slayer -Kimetsu no Yaiba- The Movie: Mugen Train' saved to Firestore.
Movie 'Kiss, Kiss!' saved to Firestore.
Movie 'Black Panther: Wakanda Forever' saved to Firestore.
Movie 'Black Adam' saved to Firestore.
Movie 'Cocaine Bear' saved to Firestore.
Movie 'AKA' saved to Firesto